# Group 3

## Document Generation

## Imports and set up LLM client

In [6]:
import sys
import os
import json
from IPython.display import display, Markdown, Code


model_name = 'claude-sonnet-4-20250514'
write_artifacts = True

try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name=model_name)


2025-10-02 10:36:57,374 ag_aisoftdev.utils INFO LLM Client configured provider=anthropic model=claude-sonnet-4-20250514 latency_ms=None artifacts_path=None


## Read in outputs from requirements generation step

In [3]:
# Set the output directory based on API provider and model
output_dir = f"../documents"
os.makedirs(output_dir, exist_ok=True)

with open(os.path.join(output_dir, "brainstormed_features.md"), "r") as f:
    brainstormed_features = f.read()

with open(os.path.join(output_dir, "user_personas.md"), "r") as f:
    user_personas = f.read()
    
with open(os.path.join(output_dir, "user_stories.json"), "r") as f:
    user_stories = f.read()

with open("../artifacts/templates/prd_template.md", "r") as f:
    prd_template = f.read()

In [7]:
simple_prd_prompt = f"""
You are a product manager for an employee onboarding tool. Generate a simple Product Requirements Document (PRD) based on user stories:

The PRD should include the following sections:
1. Introduction
2. User Personas
3. Features
4. User Stories

Please format the output as a markdown document with structured formatting, removing all ```markdown``` tags.

<User Stories>
{user_stories}
</User Stories>
"""

enhanced_prd_prompt = prompt_enhancer(simple_prd_prompt)

print("Generating Simple PRD...")
if user_stories:
    simple_prd_output = get_completion(enhanced_prd_prompt, client, model_name, api_provider)

    if write_artifacts:
        with open(os.path.join(output_dir, "simple_PRD.md"), "w", encoding="utf-8") as f:
            f.write(simple_prd_output)
    else:
        print(simple_prd_output)
        
else:
    print("Skipping PRD generation because user stories are missing.")

2025-10-02 10:39:25,510 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


Generating Simple PRD...


In [9]:
with open("../artifacts/templates/prd_template.md", "r") as f:
    prd_template_content = f.read()

template_prd_prompt = f'''You are a senior product manager. Using the following user stories, populate the provided Product Requirements Document (PRD) template.
    Also, keep in mind the output of the simple PRD you generated earlier, listed below. Use some of the relevant content from that PRD to help fill in the template, especially where it aligns with the user stories.
    In cases where the template requests information not present in the user stories, use your best judgment to fill in all reasonable details.
    User Stories: {user_stories}
    Simple PRD: {simple_prd_output}
    PRD Template: {prd_template_content}'''

print("Generating PRD from Template...")
if user_stories and prd_template_content:
    prd_from_template_output = get_completion(template_prd_prompt, client, model_name, api_provider)

    if write_artifacts:
        with open(os.path.join(output_dir, "template_PRD.md"), "w", encoding="utf-8") as f:
            f.write(prd_from_template_output)
    else:
        print(prd_from_template_output)
        
else:
    print("Skipping PRD generation because user stories or template are missing.")
    prd_from_template_output = ""

Generating PRD from Template...


In [11]:
with open("../documents/template_PRD.md", "r") as f:
    prd_template_content = f.read()

pydantic_model_prompt = f"""Generate a Pydantic model in Python for the following Product Requirements Document (PRD) using the appropriate base libraries.
- Focus only on Version 1.0 features and requirements.
- The model should be named 'ProductRequirementsDocument' and should accurately represent the structure and data types of the PRD.
- The model should use appropriate types from Python's 'typing' library (e.g., List, Optional) and have fields for each major section of the PRD.
- Each class should have a docstring explaining its purpose and usage.
- Insert a single line comment at the top of the file indicating which model and provider were used to generate the code.
<PRD>
{prd_template_content}
</PRD>"""

print("--- Generating Pydantic Model for PRD ---")
if prd_template_content:
    pydantic_model_code = get_completion(pydantic_model_prompt, client, model_name, api_provider)
    
    # Clean up the code if it's wrapped in markdown fences
    if '```' in pydantic_model_code:
        pydantic_model_code = pydantic_model_code.split('```')[1].lstrip('python').strip()
    
    print("\n--- Generated Pydantic Model ---")
    print(pydantic_model_code)

    # Save the generated Pydantic model code to a file.
    model_path = f"../artifacts/{api_provider}/{model_name}/"
    os.makedirs(model_path, exist_ok=True)
    
    with open(os.path.join(model_path, "prd_model.py"), "w", encoding="utf-8") as f:
            f.write(pydantic_model_code)
else:
    print("Skipping Pydantic model generation because template is missing.")


--- Generating Pydantic Model for PRD ---

--- Generated Pydantic Model ---
# Generated using Claude 3.5 Sonnet by Anthropic

from datetime import datetime
from typing import List, Optional, Dict, Any
from enum import Enum
from pydantic import BaseModel, Field


class DocumentStatus(str, Enum):
    """Enumeration for document status values."""
    DRAFT = "Draft"
    REVIEW = "Review"
    APPROVED = "Approved"
    FINAL = "Final"


class DocumentMetadata(BaseModel):
    """
    Represents the metadata information for the PRD document.
    Contains status, authorship, version, and update information.
    """
    status: DocumentStatus
    author: str
    version: str
    last_updated: str


class ExecutiveSummary(BaseModel):
    """
    Represents the executive summary and vision section of the PRD.
    Provides high-level overview and product vision for stakeholders.
    """
    product_description: str
    target_users: List[str]
    key_capabilities: List[str]
    vision_statement: s

In [ ]:
# TODO: Write a prompt to generate a markdown ADR template.
with open("../documents/adr_template.md", "r") as f:
    original_adr_template = f.read()
adr_template_prompt =  prompt_enhancer(f""" Generate a markdown template for an Architecture Decision Record (ADR) that includes the following sections:
1. Title
2. Status (e.g., Proposed, Accepted, Deprecated)
3. Context (the problem or forces at play)
4. Decision (the chosen solution)
5. Consequences (both positive and negative results of the decision)
Here is an example of another ADR template for reference:
{original_adr_template}
""")

print("--- Generating ADR Template ---")
adr_template_content = get_completion(adr_template_prompt, client, model_name, api_provider)

# Save the artifact
if adr_template_content:
    with open("../documents/new_adr.md", "w") as f:
        f.write(adr_template_content)
        print("--- Generated ADR Template ---")

with open("../documents/new_adr.md", "r") as f:
    new_adr_template = f.read()

new_adr_prompt = prompt_enhancer(f"""
    Research and compare database options suitable for our new hire onboarding tool that requires high 
    availability, scalability, and strong consistency. Provide a brief overview of each option, including their pros and cons, 
    and recommend the best choice based on these criteria. The options to consider are:
    1. PostgreSQL (With the pgvector extension)
    2. A Specialized Vector Database (e.g., ChromaDB, FAISS)
    Please provide a balanced view for the specific use case of our new hire onboarding tool.
    Your response should be very thorough and detailed, suitable for inclusion in an Architecture Decision Record (ADR).
    <ADR Template>
    {new_adr_template}
    </ADR Template>
    
""")

print("--- Researching Database Options ---")
db_research_output = get_completion(new_adr_prompt, client, model_name, api_provider)
if db_research_output:
    with open("../documents/db_decision_adr.md", "w") as f:
        f.write(db_research_output)
        print("--- Generated Database Decision ADR ---")